In [ ]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets
import numpy as np
#await tdmclient.notebook.start()
import import_ipynb
from PathPlanning import Global_navigation
from ekf import ExtendedKalmanFilter


speed0   = 100      # nominal speed
obstThrL = 500      # low obstacle threshold to switch state 1->0
obstThrH = 900      # high obstacle threshold to switch state 0->1
GAIN     = 10       # gain for motion control 
ERROR    = 5        # maximum angle error accepted for the robot to reach the next target 
obstSpeedGain = [0.1, 0.06, 0, -0.06, -0.1]  # gain for motor speed to avoid obstacle

pos_idx = 0
state = 0           # 0=A* algorithm, 1=obstacle avoidance
obst  = [0,0,0,0,0]  # measurements from left and right prox sensors

timer_period[0] = 10   # 10ms sampling time

class LocalNavigation:
    def __init__(self, positionX, positionY, theta, saved_pos, obst)
        self.positionX = positionX
        self.positionY = positionY
        self.theta = theta
        
        self.saved_pos = saved_pos
        self.obst = obst
        
    @tdmclient.notebook.sync_var 
    def obstacleAvoidance(self):
          """
          This function is used to detect an obstacle, not present while doing the path planning, and overcome it without colliding.
          The Thymio will continue to turn until he reaches the first target after the obstacle, from his global path planning.
        :param obst: a row vector of 5 values from the proximity sensors in front of the Thymio 
        :param position: actual postion of the robot 
        :param path: matrix composed by coordinates (x,y) of the global path planning
        :param saved_pos: index from 
        :return saved_pos_idx
        """
        global prox_horizontal, motor_left_target, motor_right_target, state, obst, obstSpeedGain, speed0 

        # acquisition from the proximity sensors to detect obstacles
        #obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]

        positionX, positionY, theta = kf.state() # get actual position of the robot

        path = PathPlanning.getPath() # get all targets from Path Planning

        # store the position from path before a local obstacle is detected
        #state, saved_pos = checkState(state, positionX, positionY, obstThrL, obstThrH)
        #saved_pos_idx = 0

        # store the index of the position from path before a local obstacle is detected
        for i in range(len(path)):
            if path[i] == self.saved_pos:
                saved_pos_idx = i  
        return saved_pos_idx
    # acquisition from the proximity sensors to detect obstacles
    def obstacleDetection():
        global prox_horizontal
        
        obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
        
    # Function to evaluate if passing the obstacle clockwise or counterclockwise
    def turningDirection(self):
        """
        Returns the direction to take to overcome the obstacle (clockwise or counterclockwise) 
        :param obst: a row vector of 5 values from the proximity sensors in front of the Thymio 
        :return turn: 1x2 vector containing the clockwise movement in the first column and the counterclockwise movement 
                      in the second column. It stores a binary value 0 if the corresponding movement is locked and 1 if unlocked
        """

        # obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
        turn = [0, 0]
        # If the obstacle detected is on the right side of the Thymio, then take the clockwise direction,
        # otherwise take the counterclockwise direction
        if (self.obst[3] + self.obst[4]) > (self.obst[0] + self.obst[1]):
            turn[0] = 1
        elif (self.obst[3] + self.obst[4]) < (self.obst[0] + self.obst[1]):
            turn[1] = 1

        return turn


    def checkState(self)
        """
        Returns the state of the Thymio which indicate whether to follow the path planning 
        from the A* algorithm or to avoid a local obstacle 
        :param position: the real time position of the Thymio  
        :param obstThrH: the value which the 5 proximity sensors exceed when they detect an object (14 cm)
        :param obstThrL: the value which indicate if the 5 proximity sensors don't detect anything 
        :return saved_pos: position of the Thymio before finding an obstacle 
        """
        global state, obstThrL, obstThrH

        position = [self.positionX, self.positionY]
        if state == 0: 
            # switch from goal tracking to obst avoidance if obstacle detected
            if (self.obst[0] > obstThrH):
                state = 1
                saved_pos = position                        
            elif (self.obst[1] > obstThrH):
                state = 1
                saved_pos = position                        
            elif (self.obst[2] > obstThrH):
                state = 1
                saved_pos = position                        
            elif (self.obst[3] > obstThrH):
                state = 1
                saved_pos = position                        
            elif (self.obst[4] > obstThrH):
                state = 1
                saved_pos = position                        

        elif state == 1:
            # switch from obst avoidance to goal tracking if obstacle got unseen
            if turn[0] == 0: 
                if turn[1] == 0: 
                    if obst[0] < obstThrL:
                        if obst[1] < obstThrL:
                            if obst[2] < obstThrL:
                                if obst[3] < obstThrL:
                                    if obst[4] < obstThrL:
                                        state = 0
                                        saved_pos = 0
    
    def currentPositionIndex(position, path):
        global pos_idx

        if position == path[pos_idx+1]
            pos_idx += 1
            
    @tdmclient.notebook.sync_var
    def motionControl(position, theta, pos_idx, path):
        global state, Speed0, GAIN, ERROR, motor_right_target, motor_left_target

        if state = 0:
            if position != path[pos_idx+1]:
                nextTarget = path[pos_idx+1]
            robotAngle = theta
            robotPosition = position
            nextTarget = path[pos_idx+1]
            current_pos_X = position[0, pos_idx]
            current_pos_Y = position[1, pos_idx]
            next_pos_X = path[0, pos_idx+1]
            next_pos_Y = path[1, pos_idx+1]
            targetAngle = np.arctan((next_pos_Y - current_pos_Y)/(next_pos_X - current_pos_X)*180/np.pi

            diff = robotAngle - targetAngle 
            if diff < ERROR:
                diff = 0
            motor_left_target = Speed0 + diff*GAIN
            motor_right_target = Speed0 - diff*GAIN
        elif state = 1:
            # adjustment for obstacles
            turn = turningDirection(obst)
            for i in range(5):
                spLeft += prox_horizontal[i] * obstSpeedGain[i] 
                spRight += prox_horizontal[i] * obstSpeedGain[4 - i] 
            # turn clockwise if proximity sensors value is below the threshold low
            if turn[0] == 1:
                if obst[3] < obstThrL:
                    if obst[4] < obstThrL:
                        spLeft = 200
                        spRight = 100
                        # continue turning until the original path is not found 
                        if position == [path[0, saved_pos_idx], path[1, saved_pos_idx]]:
                            turn[0] = 0
                        if position == [path[0, saved_pos_idx+1], path[1, saved_pos_idx+1]]:
                            turn[0] = 0
                        if position == [path[0, saved_pos_idx+2], path[1, saved_pos_idx+2]]:
                            turn[0] = 0
            # turn counterclockwise if proximity sensors value is below the threshold low
            if turn[1] == 1:
                if obst[1] < obstThrL:
                    if obst[0] < obstThrL:
                        spLeft = 100
                        spRight = 200
                        # continue turning until the original path is not found
                        if position == [path[0, saved_pos_idx], path[1, saved_pos_idx]]:
                            turn[1] = 0
                        if position == [path[0, saved_pos_idx+1], path[1, saved_pos_idx+1]]:
                            turn[1] = 0
                        if position == [path[0, saved_pos_idx+2], path[1, saved_pos_idx+2]:
                            turn[1] = 0
            # motor control
            motor_left_target = spLeft
            motor_right_target = spRight